In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE



#import minor and major regions
df = pd.concat(
    map(pd.read_csv, ['LCSA2022Summer.csv', 'LCKCL2022Summer.csv', 'LFL2022Summer.csv']), ignore_index=True)
df2 = pd.concat(
    map(pd.read_csv, ['LCS2023Spring.csv', 'LCK2023Spring.csv', 'LEC2023Winter.csv']), ignore_index=True)

#convert % columns to float 
df = df.replace('%','', regex=True)
cols = df.columns.drop(['Player', 'Team', 'Pos'])
df[cols] = df[cols].astype(float)


df = df[df['GP'] >= 9]
df2 = df2[df2['GP'] >= 9]

#find intersection (players promoted)
int_df = pd.merge(df, df2, how = 'inner', on = ['Player'])

#create list of players promoted
players = int_df['Player'].values.tolist()
print(players)


#create new column 'Promoted', set to no
df['Promoted'] = 0
#set promoted to yes if in intersection list
df['Promoted'] = np.where(df['Player'].isin(players), 1, 0)
display(df)
df = df.drop(columns=['Player', 'Team', 'Pos'])
df.dtypes
display(df)

['Busio', 'Chime', 'Eyla', 'Haeri', 'Neo', 'Solo', 'Spawn', 'Tactical', 'Tenacity', 'Tomo', 'Yeon', 'BuLLDoG', 'DnDn', 'FIESTA', 'Peter', 'Peyz', 'Photon', 'Sylvie', '113', 'Adam', 'Crownie', 'Diplex', 'Doss', 'Exakick', 'Rekkles', 'Sheo', 'Szygenda', 'Yike']


,Player,Team,Pos,GP,W%,CTR%,K,D,A,KDA,...,DPM,DMG%,D%P15,EGPM,GOLD%,STL,WPM,CWPM,WCPM,Promoted
0,Akaadian,Golden Guardians Academy,Jungle,36.0,25.0,39.0,110.0,123.0,205.0,2.6,...,367.0,16.7,18.0,209.0,19.6,5.0,0.27,0.26,0.32,0
2,AnDa,TSM Academy,Jungle,16.0,44.0,63.0,36.0,44.0,72.0,2.5,...,233.0,11.7,11.8,187.0,17.1,2.0,0.32,0.30,0.46,0
3,Armao,Team Liquid Academy,Jungle,36.0,81.0,28.0,100.0,77.0,308.0,5.3,...,365.0,13.2,13.9,218.0,16.8,3.0,0.25,0.22,0.23,0
4,Array,Golden Guardians Academy,ADC,36.0,25.0,47.0,119.0,115.0,189.0,2.7,...,543.0,23.5,24.7,262.0,23.9,0.0,0.46,0.12,0.33,0
5,Arrow,Immortals Academy,ADC,36.0,39.0,50.0,150.0,125.0,211.0,2.9,...,599.0,24.9,25.9,278.0,24.2,1.0,0.51,0.16,0.34,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Veignorem,Team GO,Support,18.0,44.0,72.0,21.0,71.0,159.0,2.5,...,172.0,8.1,6.9,121.0,10.4,0.0,1.88,0.39,0.56,0
194,Woolite,Misfits Premier,ADC,18.0,50.0,39.0,63.0,48.0,103.0,3.5,...,592.0,26.9,27.6,295.0,26.4,0.0,0.42,0.13,0.37,0
195,Xicor,Team Oplon,ADC,13.0,23.0,62.0,38.0,37.0,43.0,2.2,...,494.0,27.3,28.5,281.0,27.6,0.0,0.48,0.18,0.38,0
197,Yike,LDLC OL,Jungle,18.0,94.0,39.0,70.0,29.0,136.0,7.1,...,367.0,16.2,16.6,241.0,18.3,1.0,0.35,0.31,0.38,1


,GP,W%,CTR%,K,D,A,KDA,KP,KS%,DTH%,...,DPM,DMG%,D%P15,EGPM,GOLD%,STL,WPM,CWPM,WCPM,Promoted
0,36.0,25.0,39.0,110.0,123.0,205.0,2.6,68.9,24.1,19.1,...,367.0,16.7,18.0,209.0,19.6,5.0,0.27,0.26,0.32,0
2,16.0,44.0,63.0,36.0,44.0,72.0,2.5,61.4,20.5,19.0,...,233.0,11.7,11.8,187.0,17.1,2.0,0.32,0.30,0.46,0
3,36.0,81.0,28.0,100.0,77.0,308.0,5.3,61.0,14.9,19.5,...,365.0,13.2,13.9,218.0,16.8,3.0,0.25,0.22,0.23,0
4,36.0,25.0,47.0,119.0,115.0,189.0,2.7,67.4,26.0,17.8,...,543.0,23.5,24.7,262.0,23.9,0.0,0.46,0.12,0.33,0
5,36.0,39.0,50.0,150.0,125.0,211.0,2.9,66.6,27.7,19.8,...,599.0,24.9,25.9,278.0,24.2,1.0,0.51,0.16,0.34,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,18.0,44.0,72.0,21.0,71.0,159.0,2.5,68.2,8.0,23.7,...,172.0,8.1,6.9,121.0,10.4,0.0,1.88,0.39,0.56,0
194,18.0,50.0,39.0,63.0,48.0,103.0,3.5,70.6,26.8,20.4,...,592.0,26.9,27.6,295.0,26.4,0.0,0.42,0.13,0.37,0
195,13.0,23.0,62.0,38.0,37.0,43.0,2.2,71.1,33.3,17.5,...,494.0,27.3,28.5,281.0,27.6,0.0,0.48,0.18,0.38,0
197,18.0,94.0,39.0,70.0,29.0,136.0,7.1,69.8,23.7,17.8,...,367.0,16.2,16.6,241.0,18.3,1.0,0.35,0.31,0.38,1


In [2]:
y_col = 'Promoted'
x_cols = df.loc[:, df.columns != y_col].columns
oversample = SMOTE(random_state = 42)

training_x, testing_x, training_y, testing_y = train_test_split(df[x_cols], df[y_col], test_size=0.2, random_state = 42)
training_x, training_y = oversample.fit_resample(training_x, training_y)



In [3]:
k = 4
knnMdl = KNeighborsClassifier(n_neighbors=k).fit(training_x, training_y)
knnScores = cross_val_score(knnMdl, training_x, training_y, cv=10, scoring='accuracy')
print("KNN average accuracy on 10-fold cross validation: " + str(np.mean(knnScores)))
predictions = knnMdl.predict(testing_x)
print(predictions)
print("KNN accuracy on  test data:", accuracy_score(testing_y, predictions))

KNN average accuracy on 10-fold cross validation: 0.7948616600790512
[0 0 0 1 0 0 1 0 0 0 1 0 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0]
KNN accuracy on  test data: 0.7352941176470589


In [4]:
lrMdl = LogisticRegression(solver= 'liblinear').fit(training_x, training_y)
lrScores = cross_val_score(lrMdl, training_x, training_y, cv=10, scoring='accuracy')
print("LR average accuracy on 10-fold cross validation: " + str(np.mean(lrScores)))
lrPredictions = lrMdl.predict(testing_x)
print(predictions)
print("LR accuracy on  test data:", accuracy_score(testing_y, predictions))

LR average accuracy on 10-fold cross validation: 0.7444664031620554
[0 0 0 1 0 0 1 0 0 0 1 0 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0]
LR accuracy on  test data: 0.7352941176470589


model scores when pos = ints 0 through 4
KNN average accuracy on 10-fold cross validation: 0.817588932806324
[0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0]
KNN accuracy on  test data: 0.6764705882352942

LR average accuracy on 10-fold cross validation: 0.7403162055335969
[0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0]
LR accuracy on  test data: 0.6764705882352942

------------------------------------------------------------------------------------------------------------------

model scores when pos dropped
KNN average accuracy on 10-fold cross validation: 0.7948616600790512
[0 0 0 1 0 0 1 0 0 0 1 0 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0]
KNN accuracy on  test data: 0.7352941176470589

LR average accuracy on 10-fold cross validation: 0.7444664031620554
[0 0 0 1 0 0 1 0 0 0 1 0 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0]
LR accuracy on  test data: 0.7352941176470589

In [7]:
#PCA testing
from sklearn.decomposition import PCA
pca = PCA()
X_train = pca.fit_transform(training_x)
X_test = pca.transform(testing_x)
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

pca = PCA(n_components=2)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

[5.41545308e-01 3.18849142e-01 7.10509866e-02 4.64198056e-02
 1.00649548e-02 5.41408455e-03 2.70741960e-03 1.31491723e-03
 1.21775561e-03 7.61092607e-04 3.07369344e-04 1.37891820e-04
 6.40799132e-05 5.62247070e-05 3.75745259e-05 1.68151433e-05
 1.51596452e-05 1.04296914e-05 4.26390729e-06 2.29779010e-06
 1.28313934e-06 9.27136414e-07 1.61818254e-07 3.60389233e-08
 1.93120924e-08]
KNN average accuracy on 10-fold cross validation: 0.6956521739130435
[0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
KNN accuracy on  test data: 0.7941176470588235


In [9]:
k = 4
knnMdl = KNeighborsClassifier(n_neighbors=k).fit(X_train, training_y)
knnScores = cross_val_score(knnMdl, X_train, training_y, cv=10, scoring='accuracy')
print("KNN average accuracy on 10-fold cross validation: " + str(np.mean(knnScores)))
predictions = knnMdl.predict(X_test)
print(predictions)
print("KNN accuracy on  test data:", accuracy_score(testing_y, predictions))

KNN average accuracy on 10-fold cross validation: 0.6956521739130435
[0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
KNN accuracy on  test data: 0.7941176470588235


In [8]:
lrMdl = LogisticRegression(solver= 'liblinear').fit(X_train, training_y)
lrScores = cross_val_score(lrMdl, X_train, training_y, cv=10, scoring='accuracy')
print("LR average accuracy on 10-fold cross validation: " + str(np.mean(lrScores)))
lrPredictions = lrMdl.predict(X_test)
print(predictions)
print("LR accuracy on  test data:", accuracy_score(testing_y, predictions))

LR average accuracy on 10-fold cross validation: 0.5573122529644269
[0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
LR accuracy on  test data: 0.7941176470588235


model scores after PCA

KNN average accuracy on 10-fold cross validation: 0.6956521739130435
[0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
KNN accuracy on  test data: 0.7941176470588235

-----------------------------------------------------------------------------

LR average accuracy on 10-fold cross validation: 0.5573122529644269
[0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
LR accuracy on  test data: 0.7941176470588235